In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()  # instantiate our nn.Module
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x


In [3]:
torch.manual_seed(41)
model = Model()

In [8]:
url = 'https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'
my_df = pd.read_csv(url)

In [9]:
my_df['variety'] = my_df['variety'].replace('Setosa', 0.0)
my_df['variety'] = my_df['variety'].replace('Versicolor', 1.0)
my_df['variety'] = my_df['variety'].replace('Virginica', 2.0)
print(my_df)

     sepal.length  sepal.width  petal.length  petal.width  variety
0             5.1          3.5           1.4          0.2      0.0
1             4.9          3.0           1.4          0.2      0.0
2             4.7          3.2           1.3          0.2      0.0
3             4.6          3.1           1.5          0.2      0.0
4             5.0          3.6           1.4          0.2      0.0
..            ...          ...           ...          ...      ...
145           6.7          3.0           5.2          2.3      2.0
146           6.3          2.5           5.0          1.9      2.0
147           6.5          3.0           5.2          2.0      2.0
148           6.2          3.4           5.4          2.3      2.0
149           5.9          3.0           5.1          1.8      2.0

[150 rows x 5 columns]


C:\Users\ibzcl\AppData\Local\Temp\ipykernel_18716\3969337442.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  my_df['variety'] = my_df['variety'].replace('Virginica', 2.0)


In [11]:
# Train Test Split/ Set X, y
X = my_df.drop('variety', axis=1).values
y = my_df['variety'].values

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Convert X features to float tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

In [14]:
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [15]:
criterion = nn.CrossEntropyLoss()

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [17]:
epochs = 100
losses = []
for i in range(epochs):
    # Go forward and get a prediction
    y_pred = model.forward(X_train)

    # Measure the loss/error, gonna be high at first
    loss = criterion(y_pred, y_train)

    # Keep Track of our losses
    losses.append(loss)

    if i % 10 == 0:
        print(f'Epoch {i} and Loss: {loss}')

    # Zero gradients
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Update weights
    optimizer.step()

Epoch 0 and Loss: 1.1253553628921509
Epoch 10 and Loss: 1.0189670324325562
Epoch 20 and Loss: 0.8335332274436951
Epoch 30 and Loss: 0.6014727354049683
Epoch 40 and Loss: 0.4118161201477051
Epoch 50 and Loss: 0.2799774706363678
Epoch 60 and Loss: 0.19088388979434967
Epoch 70 and Loss: 0.13471202552318573
Epoch 80 and Loss: 0.1010771095752716
Epoch 90 and Loss: 0.08242977410554886


In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(epochs), losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')

In [19]:
with torch.no_grad():
    y_eval = model.forward(X_test)
    loss = criterion(y_eval, y_test)

In [20]:
print(f'CE Loss: {loss}')

CE Loss: 0.06505453586578369


In [21]:
# Predictions for new data
new_iris = torch.tensor([4.7, 3.2, 1.3, 0.2])
with torch.no_grad():
    print(model(new_iris))

newer_iris = torch.tensor([5.9, 3.0, 5.1, 1.8])
with torch.no_grad():
    print(model(newer_iris))

tensor([ 4.7200, -2.4076, -2.9908])
tensor([-5.9853,  4.3748,  6.3330])


In [22]:
# Save our NN Model
torch.save(model.state_dict(), 'my_really_awesome_iris_model.pt')

In [23]:
# Load the Saved Model
new_model = Model()
new_model.load_state_dict(torch.load('my_really_awesome_iris_model.pt'))
new_model.eval()

Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)

In [24]:
# Save our NN Model
torch.save(model.state_dict(), 'my_really_awesome_iris_model.pt')